# AI Finance Agent  (LlamaIndex | LlaMa-3.1)

## Table of Contents

1. **Load LLM (LlaMa-3)**
2. **Define Finance Tools**
3. **Create Finance Agent (LLM + Tools)**

## Finance Agent Functionalities

* **Stock Company Basic Information**
    * Address, Industry, Sector, business summary, website, etc.
    * Top Company Officers
* **Stock Financials**
    * Market Cap, current price, ebitda, total debt, total revenue, debt-to-equity, P/E ratio, operating margins, ebitda margins, operating cashflow, free cashflow, etc.
* **Stock Dividend & Earnings Dates**
* **Information about Mutual Fund Holders**
* **Information about Institutional Holders**
* **Stock Splits History**
* **Stock Ratings Upgrades & Downgrades by various Companies**
* **News Articles about Stock**

In [1]:
import dotenv
import os

dotenv.load_dotenv('dotenv')


groq_api_key = os.environ["GROQ_API_KEY"]

In [2]:
import yfinance as yf

yf.__version__

'0.2.41'

## 1. Load LLM

* Login to **https://console.groq.com** and create API Key.

### Groq Models

<img src="GroqModels.jpg" width=800/>

#### LlamaIndex Groq Package

* **pip install llama-index-llms-groq**

In [3]:
!pip install --upgrade llama_index langchain


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 3.3 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.6 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 2.4 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.8
    Uninstalling llama-index-core-0.11.8:
      Successfully uninstalled llama-index-core-0.11.8
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.39
    Uninstalling langchain-core-0.2.39:
      Successfully uninstalled langchain-core-0.2.39
  Attempting uninstall: langcha

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.6 requires langchain<0.3.0,>=0.2.6, but you have langchain 0.3.0 which is incompatible.
langchain-community 0.2.6 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.3.0 which is incompatible.
langchain-openai 0.1.17 requires langchain-core<0.3.0,>=0.2.20, but you have langchain-core 0.3.0 which is incompatible.


In [4]:
!pip install --upgrade llama_index

In [5]:
!pip install llama-index-llms-groq

In [6]:
from llama_index.llms.groq import Groq

llama3 = Groq(model="llama-3.1-70b-versatile", api_key=groq_api_key, temperature=0) # llama3-70b-8192

llama3

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000182B01AD4E0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x00000182D47A9AB0>, completion_to_prompt=<function default_completion_to_prompt at 0x00000182D48E52D0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama-3.1-70b-versatile', temperature=0.0, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [7]:
response = llama3.complete("Hey!! Whats up?")

print(response)

Not much, just here and ready to chat. How about you? How's your day going so far?



## 2. Define Tools

In [8]:
def flatten_dict(mapping: dict, prefix="")-> str:
    '''
    Flattens dictionary to string
    Example: 
    Input -> {'name': 'Apple', 'Address': 'CA, US', 'closePrice': 229.9}
    Output ->
    name - Apple
    Address - CA, US
    closePrice - 229.9
    '''
    total_txt = ""
    for key, val in mapping.items():
        if type(val) in [str, float, int]:
            total_txt += f"{prefix}{key} - {val}\n"
        elif isinstance(val, dict):
            total_txt += f"{key}\n"
            total_txt += flatten_dict(val, prefix="\t")
        elif isinstance(val, list):
            if isinstance(val[0], dict):
                total_txt += f"{key}\n"
                for v in val:
                    total_txt += flatten_dict(v, prefix="\t")
            else:
                total_txt += f"{key} - {val}\n"
    return total_txt

In [9]:
from llama_index.core.tools import FunctionTool
from datetime import date

def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like addrress, industry, sector, company officers, business summary, website,
       marketCap, current Price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()
    
    return flatten_dict(ticker_info)

def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    
    mf_holders = ticker_obj.get_mutualfund_holders()
    return "\n\n".join([flatten_dict(record) for record in mf_holders.to_dict(orient="records")])

def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    
    inst_holders = ticker_obj.get_institutional_holders()
    return "\n\n".join([flatten_dict(record) for record in inst_holders.to_dict(orient="records")])

def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return "\n\n".join([flatten_dict(record) for record in upgrades_downgrades.to_dict(orient="records")])

def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return flatten_dict(hist_splits.to_dict())

def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    news_links = ticker_obj.get_news()
    
    for news in news_links:
        if "thumbnail" in news:
            news.pop("thumbnail")
        if "uuid" in news:
            news.pop("uuid")
    
    return "\n\n".join([flatten_dict(news) for news in news_links])


company_information_tool = FunctionTool.from_defaults(fn=company_information)
summary_of_mutual_fund_holders_tool = FunctionTool.from_defaults(fn=summary_of_mutual_fund_holders)
summary_of_institutional_holders_tool = FunctionTool.from_defaults(fn=summary_of_institutional_holders)
stock_grade_updrages_downgrades_tool = FunctionTool.from_defaults(fn=stock_grade_updrages_downgrades)
stock_splits_history_tool = FunctionTool.from_defaults(fn=stock_splits_history)
stock_news_tool = FunctionTool.from_defaults(fn=stock_news)

tools = [
    company_information_tool,
    summary_of_mutual_fund_holders_tool,
    summary_of_institutional_holders_tool,
    stock_grade_updrages_downgrades_tool,
    stock_splits_history_tool,
    stock_news_tool
]

In [10]:
print(company_information("AAPL"))

address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - 408 996 1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as App

## 3. Define Finance Agent (LLM + Tools)

<img src="react.png" width=350/>

In [11]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

finance_agent = ReActAgent.from_tools(tools=tools, llm=llama3, verbose=True)

finance_agent

In [12]:
response = finance_agent.query("What is address of microsoft?")

print(str(response))

> Running step cacdb995-0561-45e6-92f1-3772cb45ffb0. Step input: What is address of microsoft?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'MSFT'}
Observation: address1 - One Microsoft Way
city - Redmond
state - WA
zip - 98052-6399
country - United States
phone - 425 882 8080
website - https://www.microsoft.com
industry - Software - Infrastructure
industryKey - software-infrastructure
industryDisp - Software - Infrastructure
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office lice

In [13]:
response = finance_agent.query("What is last close price of microsoft?")

print(str(response))

> Running step 5925f94e-e01e-4cb7-bb34-2d2d60236ef3. Step input: What is last close price of microsoft?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'MSFT'}
Observation: address1 - One Microsoft Way
city - Redmond
state - WA
zip - 98052-6399
country - United States
phone - 425 882 8080
website - https://www.microsoft.com
industry - Software - Infrastructure
industryKey - software-infrastructure
industryDisp - Software - Infrastructure
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Of

In [14]:
response = finance_agent.query("What is dividend yield for apple stock?")

print(str(response))

> Running step 5f36d7cc-7de8-4a09-9627-4162b1c647fd. Step input: What is dividend yield for apple stock?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'AAPL'}
Observation: address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - 408 996 1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provide

In [15]:
response = finance_agent.query("What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")

print(str(response))

> Running step 6659944d-de7e-40ab-add9-a174f3086330. Step input: What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'NVDA'}
Observation: address1 - 2788 San Tomas Expressway
city - Santa Clara
state - CA
zip - 95051
country - United States
phone - 408 486 2000
website - https://www.nvidia.com
industry - Semiconductors
industryKey - semiconductors
industryDisp - Semiconductors
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterpri

In [16]:
response = finance_agent.query("Summarize Key people at apple.")

print(str(response))

> Running step 1d87b948-b3cd-4ef9-9771-59ee213a2279. Step input: Summarize Key people at apple.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'AAPL'}
Observation: address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - 408 996 1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCa

In [17]:
response = finance_agent.query("Top mutual fund holders of microsoft stock. Share list along with their percents.")

print(str(response))

> Running step ab367932-9109-4937-ae60-ab0f7e72d30a. Step input: Top mutual fund holders of microsoft stock. Share list along with their percents.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: summary_of_mutual_fund_holders
Action Input: {'ticker': 'MSFT'}
Observation: Holder - Vanguard Total Stock Market Index Fund
pctHeld - 0.0312
Shares - 231883087
Value - 78965466446


Holder - Vanguard 500 Index Fund
pctHeld - 0.0238
Shares - 177034936
Value - 60287477105


Holder - Fidelity 500 Index Fund
pctHeld - 0.0115
Shares - 85181943
Value - 27919233637


Holder - SPDR S&P 500 ETF Trust
pctHeld - 0.0112
Shares - 83514381
Value - 26369665800


Holder - iShares Core S&P 500 ETF
pctHeld - 0.0095
Shares - 70264910
Value - 22186145332


Holder - Vanguard Growth Index Fund
pctHeld - 0.0085
Shares - 63262454
Value - 21543396085


Holder - Invesco ETF Tr-Invesco QQQ Tr, Series 1 ETF
pctHeld - 0.007900001
Shares - 58757680
Value -

In [18]:
response = finance_agent.query("Top institutional holders of apple stock and their share.")

print(str(response))

> Running step 71888c67-ccce-42ee-a2a7-a843968f6499. Step input: Top institutional holders of apple stock and their share.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: summary_of_institutional_holders
Action Input: {'ticker': 'AAPL'}
Observation: Holder - Vanguard Group Inc
pctHeld - 0.0834
Shares - 1303688506
Value - 252876459508


Holder - Blackrock Inc.
pctHeld - 0.0665
Shares - 1039640859
Value - 201659137420


Holder - Berkshire Hathaway, Inc
pctHeld - 0.0586
Shares - 915560382
Value - 177591247296


Holder - State Street Corporation
pctHeld - 0.037
Shares - 578897858
Value - 112288817516


Holder - FMR, LLC
pctHeld - 0.0196
Shares - 307066638
Value - 59561715772


Holder - Geode Capital Management, LLC
pctHeld - 0.0186
Shares - 291538165
Value - 56549657865


Holder - Price (T.Rowe) Associates Inc
pctHeld - 0.0145000005
Shares - 226650943
Value - 43963483413


Holder - Morgan Stanley
pctHeld - 0.013099999
Shar

In [19]:
response = finance_agent.query("Which firms recently changed grade of apple stock? Also, return rating changes as bullet points.")

print(str(response))

> Running step a22a8987-b434-458a-8718-588f7ef45e06. Step input: Which firms recently changed grade of apple stock? Also, return rating changes as bullet points.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: stock_grade_updrages_downgrades
Action Input: {'ticker': 'AAPL'}
Observation: Firm - Loop Capital
ToGrade - Buy
FromGrade - Hold
Action - up


Firm - Rosenblatt
ToGrade - Buy
FromGrade - Neutral
Action - up


Firm - Erste Group
ToGrade - Buy
FromGrade - Hold
Action - up


Firm - DA Davidson
ToGrade - Buy
FromGrade - Neutral
Action - up


Firm - Itau BBA
ToGrade - Market Perform
FromGrade - Underperform
Action - up


Firm - Bernstein
ToGrade - Outperform
FromGrade - Market Perform
Action - up


Firm - DZ Bank
ToGrade - Hold
FromGrade - Buy
Action - down


Firm - B of A Securities
ToGrade - Buy
FromGrade - Neutral
Action - up


Firm - Redburn Atlantic
ToGrade - Neutral
FromGrade - Buy
Action - down


Firm - Piper S

In [20]:
response = finance_agent.query("Can you share latest news articles links discussing apple stocks?")

print(str(response))

> Running step 6d81b3c1-35cd-44f1-99da-ff1c8d9ec14d. Step input: Can you share latest news articles links discussing apple stocks?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: stock_news
Action Input: {'ticker': 'AAPL'}
Observation: title - 3 Warren Buffett Stocks to Load Up on Right Now
publisher - Motley Fool
link - https://finance.yahoo.com/m/2b3857ae-3a88-3059-9f58-309f333bfa08/3-warren-buffett-stocks-to.html
providerPublishTime - 1726315200
type - STORY
relatedTickers - ['AMZN', 'AAPL', 'ULTA']


title - Apple Has a Hot New Product. It’s a Hearing Aid.
publisher - The Wall Street Journal
link - https://finance.yahoo.com/m/4aa4bc7b-42ce-3c83-95a5-7531f63e04bc/apple-has-a-hot-new-product..html
providerPublishTime - 1726275600
type - STORY
relatedTickers - ['AAPL']


title - Apple (AAPL) Stock Drops Despite Market Gains: Important Facts to Note
publisher - Zacks
link - https://finance.yahoo.com/news/apple-aapl-sto